In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch
from google.colab import drive
import os

In [ ]:
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/sarvam-transliteration-sentence"

Mounted at /content/drive


In [ ]:
df = pd.read_json("transliteration_dataset.json")
dataset = Dataset.from_pandas(df)
df_val = pd.read_json("validation_data.json")
val_dataset = Dataset.from_pandas(df_val)

In [ ]:
def format_prompt(examples):
    return {
        "text": [f"Transliterate this [{lang_code}]: {sentence}\nOutput: {output}"
                 for sentence, output, lang_code in zip(examples['input'], examples['expected_output'], examples['language'])],
        "labels": examples['expected_output']
    }

dataset = dataset.map(format_prompt, batched=True, remove_columns=["input", "expected_output", "language"])
val_dataset = val_dataset.map(format_prompt, batched=True, remove_columns=["input", "expected_output", "language"])

Map:   0%|          | 0/1097 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
model_name = "sarvamai/sarvam-1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    inputs["labels"] = tokenizer(examples["labels"], padding="max_length", truncation=True, max_length=256)["input_ids"]
    return inputs

dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text", "labels"])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["text", "labels"])

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/775k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/279M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

In [ ]:
model = get_peft_model(model, config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./sarvam-transliteration-sentence",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=5,
    logging_dir="./logs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    learning_rate=3e-5,
    fp16=True,
    save_total_limit=2,
    report_to="all"
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_dataset,
    train_dataset=dataset
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhavana0410 (abhavana0410-ssn-coe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
0,6.032400,No log
1,0.340400,No log
2,0.212500,No log
3,0.163600,No log


In [ ]:
wand db api key : 5c471cef12c538328502d01d53d802a88b28f88c

In [ ]:
trainer.save_model(output_dir)

# tokenizer = AutoTokenizer.from_pretrained("./sarvam-transliteration-sentence")
# model = AutoModelForCausalLM.from_pretrained("./sarvam-transliteration-sentence")

In [ ]:
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(68096, 2048)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=2048,

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import torch

def transliterate(sentence, lang_code="hin"):
    input_text = f"Transliterate this [{lang_code}]: {sentence}\nOutput:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    with torch.no_grad():
        output = model.generate(**inputs, max_length=100)

    return tokenizer.decode(output[0], skip_special_tokens=True).split("Output:")[-1].strip()

test_sentences = [
    ("04-07-2023 तक, ₹ 3000 खर्च किए गए।", "hin"),
    ("அவன் 10km தூரம் ஓடினான்.", "tam"),
    ("05/06/2018 அன்று ₹7500 செலவழிக்கப்பட்டது.", "tam"),


]

for sentence, lang in test_sentences:
    print(f"Input: {sentence} → Transliteration: {transliterate(sentence, lang)}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: 04-07-2023 तक, ₹ 3000 खर्च किए गए। → Transliteration: फोर सेवन टू थ्री हंड्रेड ट्वेंटी थ्री खर्च किए गए।


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: அவன் 10km தூரம் ஓடினான். → Transliteration: அவர் டென் கிலோமீட்டர் ஓடினார்.
Input: 05/06/2018 அன்று ₹7500 செலவழிக்கப்பட்டது. → Transliteration: ஃபைவ் சிக்ஸ் டூ ஹண்ட்ரெட் எய்ட் த்ரீ டூ ஹண்ட்ரெட் எய்ட் யூரோஸ் ஸ்பென்ட்.
